<a href="https://colab.research.google.com/github/ghluque/DOLAR_UVAS/blob/main/DOLAR%2CUVAS%2CUVIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backtesting y simulación en finanzas

# Dólar vs inflación en Argentina

In [ ]:
!pip install --upgrade plotly >/dev/null 2>&1 ; #Instalo Plotly para trabajar

In [ ]:
PrepareForWeb = False
#PrepareForWeb= True U: descomentar esta línea antes de guardar en Github

# Librerias


```

```





> Instalamos las librerias a utilizar



In [ ]:
import pandas as pd #U: leer y agrupar datos
import numpy as np #Para trabajar con matrices y funciones matematicas
import plotly.graph_objects as go #U: graficar con plotly
import plotly.express as px #U: graficar con plotly
import plotly #SEE: https://medium.com/@282abhishek/using-plotly-with-nbconvert-in-google-colab-96834c4f2850
if PrepareForWeb:
  plotly.offline.init_notebook_mode(connected=False)
else:
  plotly.io.renderers.default = 'colab'

In [ ]:
fecha_100 = "2023-11-01"

# DF UVAS

In [ ]:
uva_df_tables= pd.read_html('http://data.podemosaprender.org/finanzas/UVA_BCRA_2023.html') #Sacamos los datos de el siguiente link (estan en html)
uva_df= uva_df_tables[0] #A: sabemos que hay una sola!
uva_df.rename(columns={'Valor':'UVA', 'Fecha': 'fecha'}, inplace=True) #Renombramos las columnas del dataframe
uva_df.fecha= pd.to_datetime(uva_df.fecha, format='%Y-%m-%d') #Renombramos el dataframe y usamos la libreria de tiempo, cambiando el formato de fecha
uva_df.head() #Mostramos las primeras 5 filas

,fecha,UVA
0,2023-01-01,185.32
1,2023-01-02,185.32
2,2023-01-03,186.18
3,2023-01-04,186.47
4,2023-01-05,186.76


In [ ]:
uva_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   fecha   330 non-null    datetime64[ns]
 1   UVA     330 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.3 KB


In [ ]:
uva_inicial = uva_df[uva_df['fecha'] == fecha_100]['UVA'].tolist()[0]

*¿Por qué elegimos trabajar con UVAS? Sabemos que las UVAS son bonos pero se calculan casi de la misma manera que el CER, el cual refleja la inflación.*

**Para hacer una comparativa, sería conveniente pasar los valores a Indices**

In [ ]:
uva_df['indice']=uva_df['UVA'] / uva_inicial *100 #Divido toda la columna venta por el primer dia que quiero comparar
uva_df

,fecha,UVA,indice
0,2023-01-01,185.32,48.454740
1,2023-01-02,185.32,48.454740
2,2023-01-03,186.18,48.679600
3,2023-01-04,186.47,48.755425
4,2023-01-05,186.76,48.831250
...,...,...,...
325,2023-11-22,411.75,107.658317
326,2023-11-23,412.84,107.943314
327,2023-11-24,413.94,108.230926
328,2023-11-25,415.04,108.518538


*De esta forma podemos ver que el indice = dia inicial x 100/ los valores de la columna 'UVA'*



# DF Dólar

In [ ]:
usd_blue_df= pd.read_csv("https://data.podemosaprender.org/finanzas/USD_BLUE_2023.csv", sep="\t") #Sacamos los datos del link (estan en formato csv)
usd_blue_df.fecha= pd.to_datetime(usd_blue_df.fecha, format='%Y/%m/%d') #Usamos la libreria de tiempo, cambiando el formato de fecha
usd_blue_df.head() #Mostramos las primeras 5 filas

,fecha,compra,venta
0,2023-01-02,342.0,342.0
1,2023-01-03,350.0,350.0
2,2023-01-04,350.0,350.0
3,2023-01-05,349.0,349.0
4,2023-01-06,350.0,350.0


**Ahora pasamos a indice el precio de venta del dolar**



In [ ]:
usd_blue_inicial = usd_blue_df[usd_blue_df['fecha'] == fecha_100]['venta'].tolist()[0] #El primer dia que quiero comparar

usd_blue_inicial

865.0

In [ ]:
usd_blue_df['indice']=usd_blue_df['venta'] / usd_blue_inicial *100 #Divido toda la columna venta por el primer dia que quiero comparar
usd_blue_df

,fecha,compra,venta,indice
0,2023-01-02,342.0,342.0,39.537572
1,2023-01-03,350.0,350.0,40.462428
2,2023-01-04,350.0,350.0,40.462428
3,2023-01-05,349.0,349.0,40.346821
4,2023-01-06,350.0,350.0,40.462428
...,...,...,...,...
242,2023-11-17,900.0,900.0,104.046243
243,2023-11-21,1025.0,1025.0,118.497110
244,2023-11-22,1030.0,1030.0,119.075145
245,2023-11-23,995.0,995.0,115.028902


# Merge para unir DF's

**Comparamos datos con Merge**

In [ ]:
uva_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   fecha   330 non-null    datetime64[ns]
 1   UVA     330 non-null    float64       
 2   indice  330 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 7.9 KB


In [ ]:
indices_df = pd.merge(uva_df, usd_blue_df, on="fecha")
indices_df.rename(columns={'indice_x':'Indice_UVA', 'indice_y':'Indice_USD'}, inplace=True)
indices_df

,fecha,UVA,Indice_UVA,compra,venta,Indice_USD
0,2023-01-02,185.32,48.454740,342.0,342.0,39.537572
1,2023-01-03,186.18,48.679600,350.0,350.0,40.462428
2,2023-01-04,186.47,48.755425,350.0,350.0,40.462428
3,2023-01-05,186.76,48.831250,349.0,349.0,40.346821
4,2023-01-06,187.04,48.904461,350.0,350.0,40.462428
...,...,...,...,...,...,...
242,2023-11-17,406.31,106.235946,900.0,900.0,104.046243
243,2023-11-21,407.39,106.518329,1025.0,1025.0,118.497110
244,2023-11-22,411.75,107.658317,1030.0,1030.0,119.075145
245,2023-11-23,412.84,107.943314,995.0,995.0,115.028902


#Graficar con Plotly

In [ ]:
fig = px.line(indices_df, x="fecha", y=["Indice_UVA","Indice_USD"], title='Dólar vs Inflación 15-11-2023 = 100') #Graficamos definiendo las columnas del df definiendo cual es el eje x;y
fig.show()

In [ ]:
uva_usd_df= pd.merge(uva_df, usd_blue_df, on="fecha") #Renombramos el dataframe usando merge para integrar datos de distintos matrices a una. Teniendo en cuenta que la unica columna en comun es 'fecha'

uva_usd_df.head() #Mostramos las primeras 5 filas

,fecha,UVA,indice_x,compra,venta,indice_y
0,2023-01-02,185.32,48.454740,342.0,342.0,39.537572
1,2023-01-03,186.18,48.679600,350.0,350.0,40.462428
2,2023-01-04,186.47,48.755425,350.0,350.0,40.462428
3,2023-01-05,186.76,48.831250,349.0,349.0,40.346821
4,2023-01-06,187.04,48.904461,350.0,350.0,40.462428


In [ ]:
fig = px.line(uva_usd_df, x="fecha", y=["UVA","venta"], title='Dólar vs Inflación') #Graficamos definiendo las columnas del df definiendo cual es el eje x;y
fig.show()

In [ ]:
uva_usd_df['usd_en_uva']= uva_usd_df.venta / uva_usd_df.UVA
uva_usd_df.head()

,fecha,UVA,indice_x,compra,venta,indice_y,usd_en_uva
0,2023-01-02,185.32,48.454740,342.0,342.0,39.537572,1.845457
1,2023-01-03,186.18,48.679600,350.0,350.0,40.462428,1.879901
2,2023-01-04,186.47,48.755425,350.0,350.0,40.462428,1.876978
3,2023-01-05,186.76,48.831250,349.0,349.0,40.346821,1.868709
4,2023-01-06,187.04,48.904461,350.0,350.0,40.462428,1.871257


In [ ]:
uva_usd_df['fecha'] = '2022-03-01'
uva_usd_df


,fecha,UVA,indice_x,compra,venta,indice_y,usd_en_uva
0,2022-03-01,185.32,48.454740,342.0,342.0,39.537572,1.845457
1,2022-03-01,186.18,48.679600,350.0,350.0,40.462428,1.879901
2,2022-03-01,186.47,48.755425,350.0,350.0,40.462428,1.876978
3,2022-03-01,186.76,48.831250,349.0,349.0,40.346821,1.868709
4,2022-03-01,187.04,48.904461,350.0,350.0,40.462428,1.871257
...,...,...,...,...,...,...,...
242,2022-03-01,406.31,106.235946,900.0,900.0,104.046243,2.215057
243,2022-03-01,407.39,106.518329,1025.0,1025.0,118.497110,2.516017
244,2022-03-01,411.75,107.658317,1030.0,1030.0,119.075145,2.501518
245,2022-03-01,412.84,107.943314,995.0,995.0,115.028902,2.410135


In [ ]:
indice_merval= pd.read_csv("https://infra.datos.gob.ar/catalog/sspm/dataset/169/distribution/169.1/download/indices-bursatiles-argentina-indices-cotizaciones.csv", sep=",") #Sacamos los datos del link (estan en formato csv)
indice_merval.indice_tiempo= pd.to_datetime(indice_merval.indice_tiempo, format='%Y/%m/%d') #Usamos la libreria de tiempo, cambiando el formato de fecha
indice_merval.head() #Mostramos las primeras 5 filas

,indice_tiempo,bolsa_g,burcap,merval,merval_arg,merval_25
0,2004-10-04,51553.00,2881.26,1167.09,1085.43,1151.0
1,2004-10-05,50346.26,2830.79,1135.37,1054.10,1151.0
2,2004-10-06,50553.64,2852.15,1128.00,1046.48,1113.0
3,2004-10-07,50978.02,2868.16,1140.26,1059.00,1126.0
4,2004-10-08,50763.08,2853.87,1128.74,1048.06,1115.0
